In [ ]:
import os

import numpy as np
import pandas as pd
from resolve_path import ajuste_path, read_input
from unidecode import unidecode

In [ ]:
df = read_input("BD Acid_fechamento_Ago.csv")
pathInput = "data/input/"
pathInput = ajuste_path(pathInput)

# Filtrar apenas arquivos Excel que contenham "acid" no nome e tenham extensão .xlsx ou .xls
excel_files = [f for f in os.listdir(pathInput) if (
    "acid" in f.lower()) and (f.endswith('.xlsx') or f.endswith('.xls'))]

# Pegar o caminho completo dos arquivos
full_paths = [os.path.join(pathInput, f) for f in excel_files]

# Verificar se há arquivos que correspondam ao critério
if full_paths:
    # Identificar o arquivo mais recente
    latest_file = max(full_paths, key=os.path.getmtime)
    print(f"Arquivo mais recente com 'acid' no nome: {latest_file}")
else:
    print("Nenhum arquivo com 'acid' no nome foi encontrado.")

# Identificar o arquivo mais recente
latest_file = max(full_paths, key=os.path.getmtime)

latest_file_csv = latest_file.replace(".xlsx", ".csv")

df = pd.read_csv(latest_file_csv, sep="#", encoding="utf-8")

## Pré processamento das colunas:

In [ ]:
df.columns = df.columns.str.lower()
df.columns = df.columns.map(lambda x: unidecode(str(x)))
df.info()

## Pré processamento das linhas

### Utilitários

In [ ]:
nan_dict = {
    "na": np.nan,
    "nan": np.nan,
    "n.a": np.nan,
    "n.a.": np.nan,
    "n/a": np.nan,
    "-": np.nan,
}

### Empregado

In [ ]:
df["empregado"] = df["empregado"].str.lower()
df["empregado"] = df["empregado"].map(lambda x: unidecode(str(x)))
df["empregado"].unique()

### Classificação

In [ ]:
df["classificacao"] = df["classificacao"].str.lower()
df["classificacao"] = df["classificacao"].map(lambda x: unidecode(str(x)))
df["classificacao"].unique()

### Empresa

In [ ]:
df["empresa"] = df["empresa"].str.lower()
df["empresa"] = df["empresa"].map(lambda x: unidecode(str(x)))
df["empresa"].unique()

### Fornecedor

In [ ]:
df["fornecedor"] = df["fornecedor"].str.lower()
df["fornecedor"] = df["fornecedor"].map(lambda x: unidecode(str(x)))

df["fornecedor"] = df["fornecedor"].replace(nan_dict)
df["fornecedor"].unique()

### Colocando próprios com fornecedor como terceiro

In [ ]:
df.loc[df["fornecedor"].notna(), "empregado"] = "terceiro"
print(df["empregado"].unique())

### ID

In [ ]:
df["id"] = df["id"].str.lower()
df["id"] = df["id"].map(lambda x: unidecode(str(x)))

df["id"] = df["id"].str.lstrip("\t")

df["id"] = df["id"].replace(nan_dict)

indices_com_repeticao = []
print("Número de linhas antes da separação de ids agrupados: ", len(df))
for index, row in df.iterrows():
    if isinstance(row['id'], str) and ' e ' in row['id']:

        matriculas = row['id'].replace(' e ', '|')
        matriculas = matriculas.replace(', ', '|')
        matriculas = matriculas.strip()

        nomes = row['nome'].replace(' e ', '|')
        nomes = nomes.replace(', ', '|')

        lista_matriculas = matriculas.split('|')
        lista_nomes = nomes.split('|')

        for i in range(len(lista_matriculas)):
            nova_linha = row.copy()
            nova_linha['id'] = lista_matriculas[i]
            nova_linha['nome'] = lista_nomes[i]
            df = pd.concat([df, pd.DataFrame([nova_linha])], ignore_index=True)

        indices_com_repeticao.append(index)

print("Número de linhas após a separação de ids agrupados: ", len(df))
df = df.drop(indices_com_repeticao)
print("Número de linhas após a remoção de ids agrupados: ", len(df))

### Dia

In [ ]:
df["dia"].unique()

### Mes

In [ ]:
df["mes"].unique()

### Ano

In [ ]:
df["ano"].unique()

### Descrição

In [ ]:
df["descricao"].str.lower()
df["descricao"].map(lambda x: unidecode(str(x)))

df["descricao"] = df["descricao"].replace(nan_dict)

### Coordenadas

In [ ]:
def processa_coordenada(value):
    if isinstance(value, str):
        value = value.replace(",", ".")  # Substitui vírgula por ponto
        value = value.replace("°", "")  # Remove "°"
        return float(value)

    return value


def corrige_coordenada(df):
    # Corrigir longitude digitada errada (número muito grande)
    df["longitude"] = df["longitude"].apply(
        lambda x: x / 100000 if pd.notna(x) and x < -100 else x
    )
    # Corrigir sinal das coordenadas trocado
    df["longitude"] = df["longitude"].apply(
        lambda x: x * (-1) if pd.notna(x) and x > 0 else x
    )
    # Corrigir vírgulas das coordenadas digitadas errado
    df["longitude"] = df["longitude"].apply(
        lambda x: x * 10 if pd.notna(x) and x > -10 else x
    )
    # Desconsiderar pontos na Antártica
    df = df[df["latitude"].apply(lambda x: pd.notna(
        x) and x > -40 if pd.notna(x) else True)]

    return df

Latitude

In [ ]:
print(df[["latitude", "longitude"]].isna().sum())

print(df["latitude"].unique())
df["latitude"] = df["latitude"].apply(processa_coordenada)
print(df["latitude"].unique())

Longitude

In [ ]:
print(df["longitude"].unique())
df["longitude"] = df["longitude"].apply(processa_coordenada)
print(df["longitude"].unique())

In [ ]:
df = corrige_coordenada(df)
print(df[["latitude", "longitude"]].isna().sum())

### Exportar o dataset

In [ ]:
df = df.drop(columns=["unnamed: 0"])

pathUtil = "data/util/"
pathUtil = ajuste_path(pathUtil)

df.to_csv(pathUtil + "acidentes/" + "acidentes_preprocessado.csv",
          sep="#", encoding="utf-8", index=False)